# Laboratorium 3
#### Bartosz Hanc

---

1. Zaimplementuj algorytm obliczania odległości edycyjnej w taki sposób, aby możliwe było określenie
   przynajmniej jednej sekwencji edycji (dodanie, usunięcie, zmiana znaku), która pozwala w
   minimalnej liczbie kroków, przekształcić jeden łańcuch w drugi.

In [12]:
def Levenshtein_dist(s1: str, s2: str):
    m, n = len(s1), len(s2)
    s1 = " " + s1
    s2 = " " + s2
    d = [[None for _ in range(n + 1)] for _ in range(m + 1)]
    parent = [[None for _ in range(n + 1)] for _ in range(m + 1)]
    operation = [[None for _ in range(n + 1)] for _ in range(m + 1)]

    for i in range(m + 1):
        d[i][0] = i
        if i > 0:
            operation[i][0] = ("delete all", i)

    for j in range(n + 1):
        d[0][j] = j
        if j > 0:
            operation[0][j] = ("insert all", -1, s2[1:j+1])

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            cost = 0 if s1[i] == s2[j] else 1
            min_cost = min(
                d[i - 1][j] + 1,  # deletion
                d[i][j - 1] + 1,  # insertion
                d[i - 1][j - 1] + cost,  # change
            )

            if min_cost == d[i - 1][j] + 1:
                operation[i][j] = ("delete", i - 1)
                parent[i][j] = (i - 1, j)

            elif min_cost == d[i][j - 1] + 1:
                operation[i][j] = ("insert", i - 1, s2[j])
                parent[i][j] = (i, j - 1)

            elif min_cost == d[i - 1][j - 1] + 1:
                operation[i][j] = ("change", i - 1, s2[j])
                parent[i][j] = (i - 1, j - 1)

            else:
                parent[i][j] = (i - 1, j - 1)

            d[i][j] = min_cost

    steps = []
    i, j = m, n
    while parent[i][j] != None:
        if operation[i][j] != None:
            steps.append(operation[i][j])
        i, j = parent[i][j]
    if operation[i][j] != None:
        steps.append(operation[i][j])

    return d[m][n], steps[::-1]


2. Na podstawie poprzedniego punktu zaimplementuj prostą wizualizację działania algorytmu, poprzez
   wskazanie kolejnych wersji pierwszego łańcucha, w których dokonywana jest określona zmiana.
   "Wizualizacja" może działać w trybie tekstowym. Np. zmiana łańcuch "los" w "kloc" może być
   zrealizowana następująco:
    1. \*k\*los (dodanie litery k)
    2. klo\*c\* (zamiana s->c) 

In [4]:
def show_steps(s1: str, s2: str) -> None:
    dist, steps = Levenshtein_dist(s1, s2)
    s, di, n = s1, 0, 1

    print(f"{s1} -> {s2}")
    print(f"Levenshtein distance: {dist}")

    for step in steps:
        match step[0]:

            case "delete":
                i = step[1]
                print(f"{n}. Delete: {s[:i+di]}[{s[i+di]}]{s[i+di+1:]}")
                s = s[: i + di] + s[i + di + 1 :]
                di -= 1

            case "insert":
                i, char = step[1], step[2]
                print(f"{n}. Insert: {s[:i+di+1]}[{char}]{s[i+di+1:]}")
                s = s[: i + di + 1] + char + s[i + di + 1 :]
                di += 1

            case "change":
                i, char = step[1], step[2]
                print(f"{n}. Change: {s[:i+di]}[{s[i+di]}->{char}]{s[i+di+1:]}")
                s = s[: i + di] + char + s[i + di + 1 :]

            case "insert all":
                chars = step[2]
                print(f"{n}. Insert: [{chars}]{s}")
                s = chars + s
                di += len(chars)

            case "delete all":
                i = step[1]
                print(f"{n}. Delete [{s[:i]}]{s[i:]}")
                s = s[i:]
                di -= i

        n += 1

    print(f"{s} == {s2} ({s == s2})")


3. Przedstaw wynik działania algorytmu z p. 2 dla następujących par łańcuchów:
    * los - kloc
    * Łódź - Lodz
    * kwintesencja - quintessence
    * ATGAATCTTACCGCCTCG - ATGAGGCTCTGGCCCCTG

In [5]:
for s1, s2 in (
    ("los", "kloc"),
    ("Łódź", "Lodz"),
    ("kwintesencja", "quintessence"),
    ("ATGAATCTTACCGCCTCG", "ATGAGGCTCTGGCCCCTG"),
):
    show_steps(s1, s2)
    print("_" * 42)


los -> kloc
Levenshtein distance: 2
1. Insert: [k]los
2. Change: klo[s->c]
kloc == kloc (True)
__________________________________________
Łódź -> Lodz
Levenshtein distance: 3
1. Change: [Ł->L]ódź
2. Change: L[ó->o]dź
3. Change: Lod[ź->z]
Lodz == Lodz (True)
__________________________________________
kwintesencja -> quintessence
Levenshtein distance: 5
1. Change: [k->q]wintesencja
2. Change: q[w->u]intesencja
3. Insert: quintes[s]encja
4. Change: quintessenc[j->e]a
5. Delete: quintessence[a]
quintessence == quintessence (True)
__________________________________________
ATGAATCTTACCGCCTCG -> ATGAGGCTCTGGCCCCTG
Levenshtein distance: 7
1. Change: ATGA[A->G]TCTTACCGCCTCG
2. Change: ATGAG[T->G]CTTACCGCCTCG
3. Insert: ATGAGGCT[C]TACCGCCTCG
4. Change: ATGAGGCTCT[A->G]CCGCCTCG
5. Insert: ATGAGGCTCTG[G]CCGCCTCG
6. Delete: ATGAGGCTCTGGCC[G]CCTCG
7. Delete: ATGAGGCTCTGGCCCCT[C]G
ATGAGGCTCTGGCCCCTG == ATGAGGCTCTGGCCCCTG (True)
__________________________________________


4. Zaimplementuj algorytm obliczania najdłuższego wspólnego podciągu dla pary ciągów elementów.

In [6]:
def lcs(A, B):
    n, m = len(A), len(B)
    C = [[None for _ in range(m + 1)] for _ in range(n + 1)]

    for i in range(n + 1):
        C[i][0] = 0

    for i in range(m + 1):
        C[0][i] = 0

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            if A[i - 1] == B[j - 1]:
                C[i][j] = C[i - 1][j - 1] + 1
            else:
                C[i][j] = max(C[i - 1][j], C[i][j - 1])

    res = []
    i, j = n, m
    while i != 0 and j != 0:
        if C[i - 1][j] == C[i][j]:
            i, j = i - 1, j
        elif C[i][j - 1] == C[i][j]:
            i, j = i, j - 1
        else:
            res.append(A[i - 1])
            i, j = i - 1, j - 1

    return res[::-1]


5. Korzystając z gotowego tokenizera (np. spaCy - https://spacy.io/api/tokenizer) dokonaj podziału
   załączonego tekstu na tokeny.

In [7]:
from spacy.tokenizer import Tokenizer
from spacy.lang.pl import Polish

nlp = Polish()
tokenizer = Tokenizer(nlp.vocab)

with open("romeo-i-julia-700.txt", "r", encoding="utf-8") as file:
    text = file.read()
tokens = tokenizer(text)


6. Stwórz 2 wersje załączonego tekstu, w których usunięto 3% losowych tokenów.

In [8]:
from random import choices


def rm_random(tokens, p=0.03):
    rnd_tokens = set(choices(list(tokens), k=int(p * len(tokens))))
    out_tokens = []
    for token in tokens:
        if token not in rnd_tokens:
            out_tokens.append(token)

    return out_tokens


tokens1 = rm_random(tokens)
tokens2 = rm_random(tokens)

with open("text1.txt", "w", encoding="utf-8") as file:
    for token in tokens1:
        file.write(token.text_with_ws)

with open("text2.txt", "w", encoding="utf-8") as file:
    for token in tokens2:
        file.write(token.text_with_ws)


7. Oblicz długość najdłuższego podciągu wspólnych tokenów dla tych tekstów.

In [9]:
print(len(lcs(tokens1, tokens2)))

2139


8. Korzystając z algorytmu z punktu 4 skonstruuj narzędzie, o działaniu podobnym do narzędzia
   `diff`, tzn. wskazującego w dwóch plikach linie, które się różnią. Na wyjściu narzędzia powinny
   znaleźć się elementy, które nie należą do najdłuższego wspólnego podciągu. Należy wskazać, skąd
   dana linia pochodzi (< > - pierwszy/drugi plik) oraz numer linii w danym pliku.

In [10]:
def diff(file1, file2):
    with open(file1, "r", encoding="utf-8") as f:
        original = f.readlines()

    with open(file2, "r", encoding="utf-8") as f:
        new = f.readlines()

    common = lcs(original, new)

    i, j, k = 0, 0, 0
    while i < len(original) or j < len(new) or k < len(common):

        if i < len(original) and j < len(new) and k < len(common):

            if original[i] == common[k] == new[j]:
                i += 1
                j += 1
                k += 1

            elif original[i] != common[k]:
                print(f"< ({i}) {original[i]}")
                i += 1

            elif new[j] != common[k]:
                print(f"> ({j}) {new[j]}")
                j += 1

        elif i >= len(original):

            if common[k] == new[j]:
                j += 1
                k += 1

            else:
                print(f"> ({j}) {new[j]}")
                j += 1

        elif j >= len(new):

            if original[i] == common[k]:
                i += 1
                k += 1
            else:
                print(f"< ({i}) {original[i]}")
                i += 1


9. Przedstaw wynik działania narzędzia na tekstach z punktu 6. Zwróć uwagę na dodanie znaków
   przejścia do nowej linii, które są usuwane w trakcie tokenizacji.

In [11]:
diff("text1.txt", "text2.txt")

< (14)  * ROMEO — syn Montekiego

> (14)  * ROMEO — syn 

< (17)  * TYBALT — krewny Pani Kapulet* LAURENTY — ojciec franciszkanin

> (17)  * TYBALT — krewny Pani Kapulet

> (18)  * LAURENTY — ojciec franciszkanin

< (20)  * GRZEGORZ — słudzy Kapuleta

< (21)  * ABRAHAM — służący Montekiego* APTEKARZ

> (21)  * SAMSON, GRZEGORZ — słudzy Kapuleta

> (22)  * ABRAHAM — służący Montekiego

> (23)  * APTEKARZ

< (24)  * 

< (25)  * DOWÓDCA WARTY

< (26)  * PANI MONTEKI — małżonka Montekiego

> (26)  * PIOTR

> (27)  * DOWÓDCA WARTY* PANI MONTEKI — małżonka Montekiego

< (28)  * JULIA — Kapuletów

> (29)  * JULIA — córka Kapuletów

< (43) Dwa rody, zacne jednako i sławne —

< (44) Tam, gdzie się rzecz ta rozgrywa, Weronie,

> (44) Dwa rody, zacne jednako i sławne 

> (45) Tam, gdzie się rzecz ta rozgrywa, w Weronie,

< (48) Z łon tych dwu wrogów wzięło bowiem życie,

< (49) Pod najstraszliwszą gwiazd, kochanków dwoje;Po pełnym przygód nieszczęśliwych bycie

> (49) Z łon tych dwu wrogów wzięło